In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import winsound
import time
import os
import json
import random  # Added missing import for random module

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    min_detection_confidence=0.5, min_tracking_confidence=0.5
)

# Eye landmark indices (right and left eye)
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]

# Font for displaying text
font = cv2.FONT_HERSHEY_TRIPLEX

# Initialize interface with standard height (reverted from 900)
mcq_interface = np.zeros((800, 1000, 3), np.uint8)

# Orange and black theme color palette
colors = {
    "background": (10, 10, 10),       # Deep black
    "panel": (25, 25, 25),            # Dark gray
    "highlight": (0, 125, 255),       # Bright orange (BGR)
    "highlight_hover": (0, 165, 255), # Lighter orange for hover (BGR)
    "highlight_dark": (0, 80, 170),   # Darker orange
    "text": (255, 255, 255),          # White
    "text_secondary": (200, 200, 200),# Light gray
    "correct": (0, 200, 50),          # Green
    "incorrect": (50, 50, 250),       # Red (BGR)
    "neutral": (70, 70, 70),          # Dark gray
    "button": (40, 40, 40),           # Button background
    "button_border": (0, 100, 200),   # Orange border (BGR)
    "progress": (0, 140, 230)         # Orange progress (BGR)
}

# Programming MCQ questions pool
all_mcq_questions = [
    {
        "question": "Python comment symbol?",
        "options": ["//", "/* */", "#", "<!-- -->"],
        "correct": 2
    },
    {
        "question": "LIFO data structure?",
        "options": ["Queue", "Stack", "Tree", "Array"],
        "correct": 1
    },
    {
        "question": "CSS stands for?",
        "options": ["Computer Style Sheets", "Creative Style System", "Cascading Style Sheets", "Colorful Style Sheets"],
        "correct": 2
    },
    {
        "question": "NOT a JavaScript framework?",
        "options": ["React", "Angular", "Django", "Vue"],
        "correct": 2
    },
    {
        "question": "HTML largest heading element?",
        "options": ["<h1>", "<heading>", "<head>", "<h6>"],
        "correct": 0
    },
    {
        "question": "Add to Python list?",
        "options": ["add()", "append()", "insert()", "extend()"],
        "correct": 1
    },
    {
        "question": "SQL data retrieval command?",
        "options": ["GET", "EXTRACT", "SELECT", "FETCH"],
        "correct": 2
    },
    {
        "question": "API stands for?",
        "options": ["Application Programming Interface", "Automated Program Integration", "Application Process Integration", "Advanced Programming Interface"],
        "correct": 0
    },
    {
        "question": "JavaScript variable declaration?",
        "options": ["var", "let", "const", "All of these"],
        "correct": 3
    },
    {
        "question": "HTTP GET vs POST?",
        "options": ["Same thing", "Speed difference", "Request methods", "Security levels"],
        "correct": 2
    },
    {
        "question": "What creates table in SQL?",
        "options": ["CREATE TABLE", "BUILD TABLE", "MAKE TABLE", "CONSTRUCT TABLE"],
        "correct": 0
    },
    {
        "question": "Memory leak cause?",
        "options": ["Garbage collection", "Unused allocated memory", "Cache overflows", "Memory fragmentation"],
        "correct": 1
    },
    {
        "question": "Binary tree traversal method?",
        "options": ["InOrder", "PostOrder", "PreOrder", "All of these"],
        "correct": 3
    },
    {
        "question": "Database normalization purpose?",
        "options": ["Speed", "Redundancy reduction", "Security", "Size optimization"],
        "correct": 1
    },
    {
        "question": "MVC's V stands for?",
        "options": ["Virtual", "View", "Variant", "Viewport"],
        "correct": 1
    },
    {
        "question": "REST API principle?",
        "options": ["Stateful sessions", "Custom protocols", "Stateless communication", "XML only"],
        "correct": 2
    },
    {
        "question": "JSON vs XML?",
        "options": ["Different purposes", "Data formats", "Programming languages", "Protocol types"],
        "correct": 1
    },
    {
        "question": "Function vs Method difference?",
        "options": ["No difference", "Class association", "Return type", "Parameter types"],
        "correct": 1
    },
    {
        "question": "Git merge vs rebase?",
        "options": ["Same thing", "History handling", "Branch creation", "Access control"],
        "correct": 1
    },
    {
        "question": "Big O notation shows?",
        "options": ["Code size", "Execution time", "Algorithm complexity", "Memory usage"],
        "correct": 2
    }
]

# Initialize random number generator with current time
random.seed(int(time.time()))

# Randomly select 4 questions for the quiz
# Make separate selection on start (not at import time)
def select_random_questions():
    return random.sample(all_mcq_questions, 4)

# Function to calculate Eye Aspect Ratio (EAR)
def calculate_ear(eye_landmarks):
    vertical_1 = np.linalg.norm(np.array(eye_landmarks[1]) - np.array(eye_landmarks[5]))
    vertical_2 = np.linalg.norm(np.array(eye_landmarks[2]) - np.array(eye_landmarks[4]))
    horizontal = np.linalg.norm(np.array(eye_landmarks[0]) - np.array(eye_landmarks[3]))
    ear = (vertical_1 + vertical_2) / (2.0 * horizontal)
    return ear

# Helper function to draw rounded rectangle
def draw_rounded_rectangle(img, top_left, bottom_right, color, radius=10, thickness=-1):
    x1, y1 = top_left
    x2, y2 = bottom_right
    
    # Draw main rectangle
    cv2.rectangle(img, (x1+radius, y1), (x2-radius, y2), color, thickness)
    cv2.rectangle(img, (x1, y1+radius), (x2, y2-radius), color, thickness)
    
    # Draw corner circles
    cv2.circle(img, (x1+radius, y1+radius), radius, color, thickness)
    cv2.circle(img, (x2-radius, y1+radius), radius, color, thickness)
    cv2.circle(img, (x1+radius, y2-radius), radius, color, thickness)
    cv2.circle(img, (x2-radius, y2-radius), radius, color, thickness)

# Function to save quiz results to a file
def save_quiz_results(results, filename="quiz_results.json"):
    with open(filename, "w") as file:
        json.dump(results, file, indent=4)
    print(f"Results saved to {filename}")

# Function to draw the MCQ interface with orange-black theme
def draw_mcq_interface(question_index, option_index, answers):
    # Background
    mcq_interface[:] = colors["background"]
    
    # Header with orange gradient effect
    header_gradient = np.zeros((100, 1000, 3), np.uint8)
    for y in range(100):
        # Create gradient from dark to bright orange
        alpha = 1 - (y / 100)  # Gradient factor
        b = int(colors["highlight"][0])
        g = int(colors["highlight"][1] * alpha + colors["panel"][1] * (1-alpha))
        r = int(colors["highlight"][2] * alpha + colors["panel"][2] * (1-alpha))
        header_gradient[y] = (b, g, r)
    
    mcq_interface[0:100] = header_gradient
    
    # Header text with orange underline
    cv2.putText(mcq_interface, "Programming Quiz", (50, 60), font, 1.5, colors["text"], 2)
    cv2.line(mcq_interface, (50, 75), (300, 75), colors["highlight"], 3)
    
    # Question counter with modern badge
    counter_text = f"{question_index + 1}/{len(mcq_questions)}"
    text_size = cv2.getTextSize(counter_text, font, 0.9, 2)[0]
    badge_width = text_size[0] + 40
    
    draw_rounded_rectangle(mcq_interface, (1000 - badge_width - 20, 30), (980, 70), colors["highlight"], 15)
    cv2.putText(mcq_interface, counter_text, (1000 - badge_width - 20 + 20, 60), font, 0.9, colors["background"], 2)
    
    # Status area showing progress
    draw_rounded_rectangle(mcq_interface, (50, 100), (950, 150), colors["panel"], 10)
    # Orange border for progress area
    cv2.rectangle(mcq_interface, (50, 100), (950, 150), colors["highlight_dark"], 1)
    
    # Progress bar with rounded corners and orange color
    progress_width = int(((question_index) / len(mcq_questions)) * 880) if len(mcq_questions) > 0 else 0
    if progress_width > 0:
        draw_rounded_rectangle(mcq_interface, (60, 110), (60 + progress_width, 140), colors["progress"], 8)
    
    # Score display - 1-4 scale instead of 0-4
    correct_count = sum(1 for i, ans in enumerate(answers) if ans is not None and ans == mcq_questions[i]["correct"])
    # Adjust score to be on scale of 1-4 instead of 0-4
    displayed_score = correct_count if correct_count > 0 else 1
    score_text = f"Score: {displayed_score}/{len(mcq_questions)}"
    cv2.putText(mcq_interface, score_text, (70, 135), font, 0.9, colors["text"], 2)
    
    # Question panel with orange border
    draw_rounded_rectangle(mcq_interface, (50, 170), (950, 220), colors["panel"], 10)
    cv2.rectangle(mcq_interface, (50, 170), (950, 220), colors["highlight_dark"], 1)
    current_question = mcq_questions[question_index]["question"]
    cv2.putText(mcq_interface, current_question, (70, 205), font, 1, colors["text"], 2)
    
    # Blink indicator - orange progress bar
    if blink_counter > 0:
        draw_rounded_rectangle(mcq_interface, (750, 180), (950, 210), colors["neutral"], 5)
        progress = int((blink_counter / blink_frames_required) * 190)
        if progress > 10:  # Ensure we don't make a zero-width rectangle
            draw_rounded_rectangle(mcq_interface, (750, 180), (750 + progress, 210), colors["highlight"], 5)
    
    # Draw options with hover effect
    option_height = 80
    padding = 20
    options = mcq_questions[question_index]["options"]
    
    for i, option_text in enumerate(options):
        y = 250 + i * (option_height + padding)
        
        # Option label (A, B, C, D)
        option_label = chr(65 + i)  # ASCII: A=65, B=66, etc.
        
        # Simulate hover effect for the currently selected option
        if i == option_index:
            # Highlighted option with bright orange
            draw_rounded_rectangle(mcq_interface, (60, y-5), (960, y + option_height+5), colors["highlight_hover"], 10)
            font_color = colors["background"]  # Black text on orange background
            
            # Add an animated glow effect
            glow_factor = 0.3 + 0.2 * abs(np.sin(time.time() * 4))  # Pulsating glow
            overlay = mcq_interface.copy()
            draw_rounded_rectangle(overlay, (58, y-7), (962, y + option_height+7), 
                                  colors["highlight_hover"], 15)
            cv2.addWeighted(overlay, glow_factor, mcq_interface, 1-glow_factor, 0, mcq_interface)
        else:
            # Non-highlighted options with orange border
            draw_rounded_rectangle(mcq_interface, (70, y), (950, y + option_height), colors["panel"], 10)
            cv2.rectangle(mcq_interface, (70, y), (950, y + option_height), colors["highlight_dark"], 1)
            font_color = colors["text"]  # White text
        
        # Draw option text with label
        option_text_with_label = f"{option_label}. {option_text}"
        text_y = y + int(option_height / 2) + 10
        cv2.putText(mcq_interface, option_text_with_label, (100, text_y), font, 1, font_color, 2)
        
        # If this question has been answered, mark the selected option
        if question_index < len(answers) and answers[question_index] is not None:
            if i == answers[question_index]:
                # User's selection with colored circle instead of text
                circle_color = colors["correct"] if answers[question_index] == mcq_questions[question_index]["correct"] else colors["incorrect"]
                cv2.circle(mcq_interface, (920, text_y-5), 15, circle_color, -1)
            
            # Mark correct answer if different from user's selection
            if i == mcq_questions[question_index]["correct"] and answers[question_index] != i:
                cv2.circle(mcq_interface, (870, text_y-5), 15, colors["correct"], -1)
    
    # Navigation buttons at the bottom with hover effect
    button_y = 650
    
    # Previous button
    prev_highlight = option_index == len(options)
    draw_rounded_rectangle(mcq_interface, (70, button_y), (270, button_y + 50), 
                          colors["highlight_hover"] if prev_highlight else colors["button"], 10)
    # Add border to non-highlighted buttons
    if not prev_highlight:
        cv2.rectangle(mcq_interface, (70, button_y), (270, button_y + 50), colors["highlight_dark"], 1)
    cv2.putText(mcq_interface, "Previous", (120, button_y + 35), font, 1, 
               colors["background"] if prev_highlight else colors["text"], 2)
    
    # Next button
    next_highlight = option_index == len(options) + 1
    draw_rounded_rectangle(mcq_interface, (300, button_y), (500, button_y + 50), 
                          colors["highlight_hover"] if next_highlight else colors["button"], 10)
    if not next_highlight:
        cv2.rectangle(mcq_interface, (300, button_y), (500, button_y + 50), colors["highlight_dark"], 1)
    cv2.putText(mcq_interface, "Next", (380, button_y + 35), font, 1, 
               colors["background"] if next_highlight else colors["text"], 2)
    
    # Submit button
    submit_highlight = option_index == len(options) + 2
    draw_rounded_rectangle(mcq_interface, (530, button_y), (730, button_y + 50), 
                          colors["highlight_hover"] if submit_highlight else colors["button"], 10)
    if not submit_highlight:
        cv2.rectangle(mcq_interface, (530, button_y), (730, button_y + 50), colors["highlight_dark"], 1)
    cv2.putText(mcq_interface, "Submit", (580, button_y + 35), font, 1, 
               colors["background"] if submit_highlight else colors["text"], 2)
    
    # Add hover animation to highlighted button
    if option_index >= len(options):
        # Determine which button is highlighted
        if option_index == len(options):  # Previous
            button_x, button_width = 70, 200
        elif option_index == len(options) + 1:  # Next
            button_x, button_width = 300, 200
        else:  # Submit
            button_x, button_width = 530, 200
            
        # Animated border
        glow_factor = 0.4 + 0.3 * abs(np.sin(time.time() * 5))
        border_thickness = 2 + int(2 * abs(np.sin(time.time() * 5)))
        overlay = mcq_interface.copy()
        cv2.rectangle(overlay, (button_x-3, button_y-3), 
                     (button_x + button_width+3, button_y + 50+3), 
                     colors["highlight_hover"], border_thickness)
        cv2.addWeighted(overlay, glow_factor, mcq_interface, 1-glow_factor, 0, mcq_interface)
    
    # Footer with instructions in orange highlight
    cv2.putText(mcq_interface, "Blink to select the highlighted option", 
               (350, 750), font, 0.8, colors["highlight"], 1)

# Function to draw the results screen with orange-black theme
def draw_results_screen(answers):
    # Background 
    mcq_interface[:] = colors["background"]
    
    # Header panel with orange gradient
    header_gradient = np.zeros((100, 1000, 3), np.uint8)
    for y in range(100):
        # Create gradient from dark to bright orange
        alpha = 1 - (y / 100)  # Gradient factor
        b = int(colors["highlight"][0])
        g = int(colors["highlight"][1] * alpha + colors["panel"][1] * (1-alpha))
        r = int(colors["highlight"][2] * alpha + colors["panel"][2] * (1-alpha))
        header_gradient[y] = (b, g, r)
    
    mcq_interface[0:100] = header_gradient
    
    # Title with orange underline
    cv2.putText(mcq_interface, "Programming Quiz Results", (320, 60), font, 1.5, colors["text"], 2)
    cv2.line(mcq_interface, (320, 75), (680, 75), colors["highlight"], 3)
    
    # Calculate score
    correct_count = sum(1 for i, ans in enumerate(answers) if ans is not None and ans == mcq_questions[i]["correct"])
    total_questions = len(mcq_questions)
    # Adjust score to be on scale of 1-4 instead of 0-4
    displayed_score = correct_count if correct_count > 0 else 1
    percentage = (correct_count / total_questions) * 100 if total_questions > 0 else 0
    
    # Score panel with orange accent
    draw_rounded_rectangle(mcq_interface, (250, 110), (750, 160), colors["panel"], 15)
    cv2.rectangle(mcq_interface, (250, 110), (750, 160), colors["highlight_dark"], 2)
    
    # Display overall score
    score_text = f"Final Score: {displayed_score}/{total_questions} ({percentage:.1f}%)"
    cv2.putText(mcq_interface, score_text, (300, 145), font, 1, colors["text"], 2)
    
    # Results panel background with orange border - INCREASED HEIGHT
    draw_rounded_rectangle(mcq_interface, (50, 180), (950, 680), colors["panel"], 15)
    cv2.rectangle(mcq_interface, (50, 180), (950, 680), colors["highlight_dark"], 2)
    
    # Display results for each question with appropriate spacing for 4 questions
    y_pos = 220
    for i, question in enumerate(mcq_questions):
        # Question box with alternating shades and orange accents
        if i % 2 == 0:
            draw_rounded_rectangle(mcq_interface, (70, y_pos-20), (930, y_pos+100), 
                                  (colors["panel"][0]+10, colors["panel"][1]+10, colors["panel"][2]+10), 8)
            # Add subtle orange left border
            cv2.rectangle(mcq_interface, (70, y_pos-20), (75, y_pos+100), colors["highlight_dark"], -1)
        
        # Question number and text (truncated if too long)
        question_text = question["question"]
        if len(question_text) > 50:
            question_text = question_text[:47] + "..."
        
        # Question number in orange
        cv2.putText(mcq_interface, f"Q{i+1}:", (90, y_pos), font, 0.8, colors["highlight"], 2)
        # Question text in white
        cv2.putText(mcq_interface, f" {question_text}", (130, y_pos), font, 0.8, colors["text"], 2)
        
        # User's answer
        if i < len(answers) and answers[i] is not None:
            user_answer = question["options"][answers[i]]
            correct_answer = question["options"][question["correct"]]
            
            # Display user's answer without overlapping
            if answers[i] == question["correct"]:
                # Green circle for correct answer
                cv2.circle(mcq_interface, (115, y_pos+30), 12, colors["correct"], -1)
                # All text in white
                cv2.putText(mcq_interface, f"Your answer: {user_answer}", (170, y_pos + 30), font, 0.8, colors["text"], 2)
                
            else:
                # Red circle for wrong answer
                cv2.circle(mcq_interface, (115, y_pos+30), 12, colors["incorrect"], -1)
                # All text in white
                cv2.putText(mcq_interface, f"Your answer: {user_answer}", (170, y_pos + 30), font, 0.8, colors["text"], 2)
                
                # Show "Correct answer:" text in white with proper spacing
                cv2.putText(mcq_interface, "Correct answer:", (170, y_pos + 60), font, 0.8, colors["text"], 2)
                # Put the correct answer in green with space between label and answer
                cv2.putText(mcq_interface, f"{correct_answer}", (370, y_pos + 60), font, 0.8, colors["correct"], 2)
                
        else:
            # Not answered - with orange accent
            cv2.circle(mcq_interface, (115, y_pos+30), 12, colors["highlight_dark"], -1)
            cv2.putText(mcq_interface, "Not answered", 
                        (170, y_pos + 30), font, 0.8, colors["text"], 2)
        
        # Spacing between questions - works well for 4 questions
        y_pos += 110
    
    # Restart button and related elements - reverted to original position
    t = time.time()
    pulse = abs(np.sin(t * 3)) * 20  # Pulsating value between 0 and 20
    
    # Inner button with orange highlight
    draw_rounded_rectangle(mcq_interface, (350, 700), (650, 750), 
                          (colors["highlight"][0], 
                           colors["highlight"][1]+int(pulse/2), 
                           colors["highlight"][2]+int(pulse)), 15)
    
    # Animated glow effect around button
    glow_factor = 0.3 + 0.2 * abs(np.sin(time.time() * 4))
    overlay = mcq_interface.copy()
    draw_rounded_rectangle(overlay, (345, 695), (655, 755), colors["highlight_hover"], 20)
    cv2.addWeighted(overlay, glow_factor, mcq_interface, 1-glow_factor, 0, mcq_interface)
    
    # Button text
    cv2.putText(mcq_interface, "Restart Quiz", (425, 735), font, 1, colors["background"], 2)
    
    # Instruction text with orange highlight
    cv2.putText(mcq_interface, "Blink to restart the quiz", 
               (380, 780), font, 0.8, colors["highlight"], 1)

# Variables for tracking
frames = 0
question_index = 0
option_index = 0
current_mode = "quiz"  # Can be "quiz" or "results"
blink_counter = 0
blink_threshold = 0.2  # EAR threshold for blink detection
blink_frames_required = 5  # Frames required to confirm a blink

# Select random questions at startup 
mcq_questions = select_random_questions()
answers = [None] * len(mcq_questions)  # Store user's answers

# Set iterator speed (higher number = slower iteration)
iterator_speed = 30

# Initialize camera
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.flip(frame, 1)  # Flip for a mirror effect
    
    # Convert to RGB for MediaPipe
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)
    
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Get eye landmarks
            left_eye = [(face_landmarks.landmark[i].x * frame.shape[1], face_landmarks.landmark[i].y * frame.shape[0]) 
                        for i in LEFT_EYE]
            right_eye = [(face_landmarks.landmark[i].x * frame.shape[1], face_landmarks.landmark[i].y * frame.shape[0]) 
                         for i in RIGHT_EYE]
            
            # Calculate EAR
            left_ear = calculate_ear(left_eye)
            right_ear = calculate_ear(right_eye)
            avg_ear = (left_ear + right_ear) / 2
            
            # Draw eye landmarks with orange theme
            for point in left_eye + right_eye:
                # Outer orange circle
                cv2.circle(frame, (int(point[0]), int(point[1])), 4, colors["highlight_hover"][::-1], -1)
                # Inner white dot
                cv2.circle(frame, (int(point[0]), int(point[1])), 2, (255, 255, 255), -1)
            
            # Blink detection logic
            if avg_ear < blink_threshold:
                blink_counter += 1
                # Orange visual feedback for blink detection
                cv2.putText(frame, "BLINK DETECTED", (30, 30), font, 1, colors["highlight_hover"][::-1], 2)
                # Add orange tint when blinking
                overlay = frame.copy()
                cv2.rectangle(overlay, (0, 0), (frame.shape[1], frame.shape[0]), colors["highlight_dark"][::-1], -1)
                cv2.addWeighted(overlay, 0.1, frame, 0.9, 0, frame)
            else:
                if blink_counter >= blink_frames_required:
                    # Selection confirmed with a blink
                    
                    # Play sound for feedback
                    try:
                        winsound.PlaySound("SystemExclamation", winsound.SND_ALIAS)
                    except:
                        # If winsound fails or not on Windows
                        pass
                    
                    # Handle selection based on current mode
                    if current_mode == "quiz":
                        options = mcq_questions[question_index]["options"]
                        
                        if option_index < len(options):
                            # Selected one of the answer options
                            answers[question_index] = option_index
                            
                        elif option_index == len(options):  # Previous button
                            if question_index > 0:
                                question_index -= 1
                                option_index = 0
                        
                        elif option_index == len(options) + 1:  # Next button
                            if question_index < len(mcq_questions) - 1:
                                question_index += 1
                                option_index = 0
                        
                        elif option_index == len(options) + 2:  # Submit button
                            # Switch to results screen
                            current_mode = "results"
                            option_index = 0
                            # Save results
                            correct_count = sum(1 for i, ans in enumerate(answers) if ans is not None and ans == mcq_questions[i]["correct"])
                            # Adjust score to be on scale of 1-4 instead of 0-4 for saved results
                            displayed_score = correct_count if correct_count > 0 else 1
                            save_quiz_results({
                                "questions": mcq_questions,
                                "answers": answers,
                                "score": displayed_score,
                                "total": len(mcq_questions),
                                "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
                            })
                    
                    elif current_mode == "results":
                        # In results screen, any blink restarts the quiz
                        question_index = 0
                        option_index = 0
                        # Generate new random questions when restarting
                        mcq_questions = select_random_questions()
                        answers = [None] * len(mcq_questions)
                        current_mode = "quiz"
                
                blink_counter = 0  # Reset blink counter
    
    # Show blink counter on frame when counting with orange theme
    if blink_counter > 0:
        # Draw orange progress bar for blink
        progress_width = int((blink_counter / blink_frames_required) * 120)
        cv2.rectangle(frame, (frame.shape[1] - 150, 40), (frame.shape[1] - 30, 55), (40, 40, 40), -1)
        cv2.rectangle(frame, (frame.shape[1] - 150, 40), 
                     (frame.shape[1] - 150 + progress_width, 55), colors["highlight_hover"][::-1], -1)
        
        cv2.putText(frame, f"Blink: {blink_counter}/{blink_frames_required}", 
                   (frame.shape[1] - 150, 30), font, 0.7, colors["highlight_hover"][::-1], 2)
    
    # Iterate through options automatically
    if frames % iterator_speed == 0:  # Adjust speed of selection
        if current_mode == "quiz":
            options = mcq_questions[question_index]["options"]
            option_index = (option_index + 1) % (len(options) + 3)  # +3 for previous, next, and submit buttons
        # No iteration needed in results mode - restart is the only option
    
    frames += 1
    
    # Add an orange frame around camera view
    camera_height, camera_width = frame.shape[:2]
    border_color = colors["highlight"][::-1]  # Convert BGR to RGB for display
    border_thickness = 8
    
    # Add animated border - pulsating effect
    border_pulse = int(4 * abs(np.sin(time.time() * 3)))
    thickness = border_thickness + border_pulse
    
    # Top border
    cv2.rectangle(frame, (0, 0), (camera_width, thickness), border_color, -1)
    # Bottom border
    cv2.rectangle(frame, (0, camera_height-thickness), (camera_width, camera_height), border_color, -1)
    # Left border
    cv2.rectangle(frame, (0, 0), (thickness, camera_height), border_color, -1)
    # Right border
    cv2.rectangle(frame, (camera_width-thickness, 0), (camera_width, camera_height), border_color, -1)
    
    # Draw the appropriate interface
    if current_mode == "quiz":
        draw_mcq_interface(question_index, option_index, answers)
    elif current_mode == "results":
        draw_results_screen(answers)
    
    # Show windows
    cv2.imshow("Camera", frame)
    cv2.imshow("Quiz", mcq_interface)
    
    key = cv2.waitKey(1)
    if key == 27:  # Press ESC to exit
        break

cap.release()
cv2.destroyAllWindows()